In [6]:
import torch
import ase.io
from tqdm import tqdm
import os

from mace.calculators import MACECalculator

if __name__ == '__main__':
    # --- KONFIGURACJA ---
    MACE_OFF_MODEL_PATH = 'MACE-OFF24_medium.model' # Model wysokiej dokładności
    MP0_MODEL_PATH = 'MACE-MP_small.model'              # Model bazowy (baseline)
    
    INPUT_TRAJECTORY_FILE = 'mace_off_trajectory.xyz'
    OUTPUT_DATASET_FILE = 'delta_learning_dataset.xyz'
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Używane urządzenie: {device}")

    # Sprawdzenie, czy pliki modeli istnieją
    for path in [MACE_OFF_MODEL_PATH, MP0_MODEL_PATH, INPUT_TRAJECTORY_FILE]:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Nie znaleziono wymaganego pliku: '{path}'")

    # --- Krok 1: Inicjalizacja obu kalkulatorów ---
    print("Inicjalizacja kalkulatorów dla modeli mace-off i mp-0...")
    calc_mace_off = MACECalculator(model_path=MACE_OFF_MODEL_PATH, device=device)
    calc_mp0 = MACECalculator(model_path=MP0_MODEL_PATH, device=device)

    # --- Krok 2: Wczytanie trajektorii ---
    print(f"Wczytywanie trajektorii z pliku: '{INPUT_TRAJECTORY_FILE}'")
    trajectory = ase.io.read(INPUT_TRAJECTORY_FILE, index=':')
    print(f"Znaleziono {len(trajectory)} klatek do przetworzenia.")

    # --- Krok 3: Obliczenie energii i delty dla każdej klatki ---
    processed_atoms_list = []
    for atoms in tqdm(trajectory, desc="Przetwarzanie klatek"):
        # Oblicz energię z modelu bazowego (mp-0)
        atoms.calc = calc_mp0
        energy_mp0 = atoms.get_potential_energy()

        # Oblicz energię z modelu docelowego (mace-off)
        atoms.calc = calc_mace_off
        energy_mace_off = atoms.get_potential_energy()

        # Oblicz i zapisz deltę
        delta_energy = energy_mace_off - energy_mp0
        
        # Zapisz wszystkie informacje w obiekcie Atoms
        # Klucz 'energy' jest najważniejszy - to będzie nasz cel (target) w treningu
        atoms.info['delta_energy'] = delta_energy
        atoms.info['energy_mp0'] = energy_mp0
        atoms.info['energy_mace_off'] = energy_mace_off

        # --- WAŻNA ZMIANA: Odłącz kalkulator przed zapisem ---
        atoms.calc = None
        # ---------------------------------------------------
        
        processed_atoms_list.append(atoms)


    # --- Krok 4: Zapisz przetworzony zbiór danych ---
    print(f"Zapisywanie przetworzonego zbioru danych do pliku: '{OUTPUT_DATASET_FILE}'")
    ase.io.write(OUTPUT_DATASET_FILE, processed_atoms_list)

    print("\nGotowe! Zbiór danych do uczenia delta-learning został pomyślnie utworzony.")


/home/krystynasyty/myenv/lib/python3.12/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Używane urządzenie: cpu
Inicjalizacja kalkulatorów dla modeli mace-off i mp-0...
Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.


/home/krystynasyty/myenv/lib/python3.12/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Wczytywanie trajektorii z pliku: 'mace_off_trajectory.xyz'
Znaleziono 101 klatek do przetworzenia.


Przetwarzanie klatek: 100%|██████████| 101/101 [02:55<00:00,  1.74s/it]

Zapisywanie przetworzonego zbioru danych do pliku: 'delta_learning_dataset.xyz'

Gotowe! Zbiór danych do uczenia delta-learning został pomyślnie utworzony.


In [5]:
from ase import Atoms
from mace.calculators import MACECalculator
atoms = Atoms("C")

MACE_OFF_MODEL_PATH = 'MACE-OFF24_medium.model' # Model wysokiej dokładności
MP0_MODEL_PATH = 'MACE-MP_small.model'              # Model bazowy (baseline)
    

calc_mace_off = MACECalculator(model_path=MACE_OFF_MODEL_PATH, device="cpu")
calc_mp0 = MACECalculator(model_path=MP0_MODEL_PATH, device="cpu")

# Oblicz energię z modelu bazowego (mp-0)
atoms.calc = calc_mp0
print(atoms.get_potential_energy())
        # Oblicz energię z modelu docelowego (mace-off)
atoms.calc = calc_mace_off
print( atoms.get_potential_energy())



/home/krystynasyty/myenv/lib/python3.12/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)
/home/krystynasyty/myenv/lib/python3.12/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
-1.9194310604571392
-1030.5671648271828
